<a href="https://colab.research.google.com/github/rajeshwari179/summer_research/blob/main/MAPPING.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

L1 regularization + spatial loss function

In [ ]:
import numpy as np
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Flatten
from tensorflow.keras.regularizers import l1
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.datasets import cifar10

(x_train, y_train), (x_test, y_test) = cifar10.load_data()

#grayscale
def rgb_to_grayscale(images):
    return np.dot(images[..., :3], [0.2989, 0.587, 0.114])

x_train_gray = rgb_to_grayscale(x_train)
x_test_gray = rgb_to_grayscale(x_test)

# Normalize
x_train_gray = x_train_gray / 255.0
x_test_gray = x_test_gray / 255.0

x_train_gray = np.expand_dims(x_train_gray, -1)
x_test_gray = np.expand_dims(x_test_gray, -1)

model = Sequential([
    Flatten(input_shape=(32, 32, 1)),
    Dense(1024, kernel_regularizer=l1(0.08)),
    Dense(512, activation='relu', kernel_regularizer=l1(0.01)),
    Dense(512, activation='relu', kernel_regularizer=l1(0.01)),
    Dense(256, activation=None)
])

model.compile(optimizer=Adam(), loss='sparse_categorical_crossentropy', metrics=['accuracy'])
model.summary()

model.fit(x_train_gray, y_train, epochs=5, validation_split=0.1)

# Function to manually prune weights
def prune_weights(model, threshold):
    for layer in model.layers:
        if isinstance(layer, Dense):
            weights, biases = layer.get_weights()
            new_weights = np.where(np.abs(weights) < threshold, 0, weights)
            layer.set_weights([new_weights, biases])


threshold_value = 0.00008
prune_weights(model, threshold_value)

model.compile(optimizer=Adam(), loss='sparse_categorical_crossentropy' , metrics=['accuracy'])

# Evaluate the pruned model
loss, accuracy = model.evaluate(x_test_gray, y_test)
print(f"Pruned model - Loss: {loss}, Accuracy: {accuracy}")

# Function to print model weights
def print_model_weights(model):
    for layer in model.layers:
        weights = layer.get_weights()
        if weights:  # Check if there are weights to print
            print(f"Layer: {layer.name}")
            print("Weights:")
            print(weights[0])
            if len(weights) > 1:  # Check if biases exist
                print("Biases:")
                print(weights[1])
            print("\n" + "="*50 + "\n")

print_model_weights(model)


In [ ]:
import numpy as np
import torch
from scipy.spatial.distance import pdist, squareform

def spatial_loss(outputs):
    num_neurons = outputs.size(1)
    outputs_np = outputs.detach().cpu().numpy()

    # Calculate pairwise correlations
    pairwise_correlations = np.zeros((num_neurons, num_neurons))
    for i in range(num_neurons):
        for j in range(i + 1, num_neurons):
            r = np.corrcoef(outputs_np[:, i], outputs_np[:, j])[0, 1]
            pairwise_correlations[i, j] = pairwise_correlations[j, i] = r

    # Calculate the grid size and ensure it is a perfect square
    grid_size = int(np.sqrt(num_neurons))
    if grid_size * grid_size != num_neurons:
        raise ValueError("Number of neurons must be a perfect square to form a grid.")

    # Calculate distances in the grid
    distances = np.zeros((num_neurons, num_neurons))
    for i in range(grid_size):
        for j in range(grid_size):
            for k in range(grid_size):
                for l in range(grid_size):
                    index_1 = i * grid_size + j
                    index_2 = k * grid_size + l
                    distance = np.sqrt((i - k)**2 + (j - l)**2)
                    distances[index_1, index_2] = 1 / (distance + 1)

    # Compute the spatial loss
    SL = np.mean(np.abs(pairwise_correlations - distances))

    return SL


Normal model with l1 regularization, with 50 epochs


In [2]:
import torch
import torch.nn as nn
import torch.optim as optim
from torchvision import datasets, transforms

# Define the fully connected network with batch normalization and dropout
class FullyConnectedNet(nn.Module):
    def __init__(self):
        super(FullyConnectedNet, self).__init__()
        self.fc1 = nn.Linear(32 * 32, 1024)
        self.bn1 = nn.BatchNorm1d(1024)
        self.fc2 = nn.Linear(1024, 512)
        self.bn2 = nn.BatchNorm1d(512)
        self.fc3 = nn.Linear(512, 256)
        self.bn3 = nn.BatchNorm1d(256)
        self.fc4 = nn.Linear(256, 10)
        self.dropout = nn.Dropout(0.5)

    def forward(self, x):
        x = x.view(-1, 32 * 32)  # Flatten the input
        x = torch.relu(self.bn1(self.fc1(x)))
        x = self.dropout(x)
        x = torch.relu(self.bn2(self.fc2(x)))
        x = self.dropout(x)
        x = torch.relu(self.bn3(self.fc3(x)))
        x = self.dropout(x)
        x = self.fc4(x)
        return x

# Data augmentation and normalization
transform = transforms.Compose([
    transforms.Grayscale(num_output_channels=1),
    transforms.RandomHorizontalFlip(),
    transforms.RandomCrop(32, padding=4),
    transforms.ToTensor(),
    transforms.Normalize((0.5,), (0.5,))
])

train_dataset = datasets.CIFAR10(root='./data', train=True, download=True, transform=transform)
train_loader = torch.utils.data.DataLoader(train_dataset, batch_size=64, shuffle=True)

test_dataset = datasets.CIFAR10(root='./data', train=False, download=True, transform=transform)
test_loader = torch.utils.data.DataLoader(test_dataset, batch_size=64, shuffle=False)

# Initialize the model, loss function, and optimizer
model = FullyConnectedNet()
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)

# Function to apply L1 regularization
def l1_regularization(model, lambda_l1):
    l1_norm = sum(p.abs().sum() for p in model.parameters())
    return lambda_l1 * l1_norm

# Training the model
num_epochs = 50
lambda_l1 = 0.0001

for epoch in range(num_epochs):
    model.train()
    total_loss = 0
    for data, target in train_loader:
        optimizer.zero_grad()
        output = model(data)
        loss = criterion(output, target)
        loss += l1_regularization(model, lambda_l1)
        loss.backward()
        optimizer.step()
        total_loss += loss.item()

    print(f'Epoch {epoch+1}/{num_epochs}, Loss: {total_loss/len(train_loader)}')

# Testing the model
model.eval()
correct = 0
total = 0

with torch.no_grad():
    for data, target in test_loader:
        output = model(data)
        _, predicted = torch.max(output.data, 1)
        total += target.size(0)
        correct += (predicted == target).sum().item()

print(f'Accuracy: {100 * correct / total}%')


Files already downloaded and verified
Files already downloaded and verified
Epoch 1/50, Loss: 3.7751694045713187
Epoch 2/50, Loss: 3.1664483251474094
Epoch 3/50, Loss: 3.0900138491559823
Epoch 4/50, Loss: 3.0332802595080013
Epoch 5/50, Loss: 2.9485762351004365
Epoch 6/50, Loss: 2.833741909707599
Epoch 7/50, Loss: 2.7150920416083175
Epoch 8/50, Loss: 2.6299577472764817
Epoch 9/50, Loss: 2.5697404929744008
Epoch 10/50, Loss: 2.5226876107628082
Epoch 11/50, Loss: 2.4918732143119167
Epoch 12/50, Loss: 2.4637931627995524
Epoch 13/50, Loss: 2.4444908976859754
Epoch 14/50, Loss: 2.4264493641036244
Epoch 15/50, Loss: 2.41758079449539
Epoch 16/50, Loss: 2.3990146150369474
Epoch 17/50, Loss: 2.3904292126140936
Epoch 18/50, Loss: 2.376588818362302
Epoch 19/50, Loss: 2.3724924359480135
Epoch 20/50, Loss: 2.3646449237833242
Epoch 21/50, Loss: 2.354778215403447
Epoch 22/50, Loss: 2.351242602633698
Epoch 23/50, Loss: 2.3418199723333957
Epoch 24/50, Loss: 2.3367109140166846
Epoch 25/50, Loss: 2.337954

L1 regularization with spatial loss


In [5]:
import torch
import torch.nn as nn
import torch.optim as optim
from torchvision import datasets, transforms

# Define the fully connected network with batch normalization and dropout
class FullyConnectedNet(nn.Module):
    def __init__(self):
        super(FullyConnectedNet, self).__init__()
        self.fc1 = nn.Linear(32 * 32, 1024)
        self.bn1 = nn.BatchNorm1d(1024)
        self.fc2 = nn.Linear(1024, 512)
        self.bn2 = nn.BatchNorm1d(512)
        self.fc3 = nn.Linear(512, 256)
        self.bn3 = nn.BatchNorm1d(256)
        self.fc4 = nn.Linear(256, 10)
        self.dropout = nn.Dropout(0.5)

    def forward(self, x):
        x = x.view(-1, 32 * 32)  # Flatten the input
        x = torch.relu(self.bn1(self.fc1(x)))
        x = self.dropout(x)
        x = torch.relu(self.bn2(self.fc2(x)))
        x = self.dropout(x)
        x = torch.relu(self.bn3(self.fc3(x)))
        x = self.dropout(x)
        x = self.fc4(x)
        return x

# Data augmentation and normalization
transform = transforms.Compose([
    transforms.Grayscale(num_output_channels=1),
    transforms.RandomHorizontalFlip(),
    transforms.RandomCrop(32, padding=4),
    transforms.ToTensor(),
    transforms.Normalize((0.5,), (0.5,))
])

train_dataset = datasets.CIFAR10(root='./data', train=True, download=True, transform=transform)
train_loader = torch.utils.data.DataLoader(train_dataset, batch_size=64, shuffle=True)

test_dataset = datasets.CIFAR10(root='./data', train=False, download=True, transform=transform)
test_loader = torch.utils.data.DataLoader(test_dataset, batch_size=64, shuffle=False)

# Initialize the model, loss function, and optimizer
model = FullyConnectedNet()
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)

# Function to apply L1 regularization
def l1_regularization(model, lambda_l1):
    l1_norm = sum(p.abs().sum() for p in model.parameters())
    return lambda_l1 * l1_norm

# Customized spatial loss function
def spatial_loss(outputs):
    num_neurons = outputs.size(1)

    # Calculate pairwise correlations
    outputs_centered = outputs - outputs.mean(dim=0)
    pairwise_correlations = torch.matmul(outputs_centered.T, outputs_centered) / (outputs_centered.size(0) - 1)
    pairwise_correlations /= (outputs_centered.std(dim=0).unsqueeze(1) * outputs_centered.std(dim=0).unsqueeze(0) + 1e-8)

    # Calculate the grid size and ensure it is a perfect square
    grid_size = int(torch.sqrt(torch.tensor(num_neurons, dtype=torch.float32)))
    if grid_size * grid_size != num_neurons:
        raise ValueError("Number of neurons must be a perfect square to form a grid.")

    # Calculate distances in the grid
    distances = torch.zeros((num_neurons, num_neurons))
    for i in range(grid_size):
        for j in range(grid_size):
            for k in range(grid_size):
                for l in range(grid_size):
                    index_1 = i * grid_size + j
                    index_2 = k * grid_size + l
                    distance = torch.sqrt(torch.tensor((i - k)**2 + (j - l)**2, dtype=torch.float32))
                    distances[index_1, index_2] = 1 / (distance + 1)

    # Compute the spatial loss
    SL = torch.mean(torch.abs(pairwise_correlations - distances))

    return SL

# Training the model
num_epochs = 50
lambda_l1 = 0.0001
lambda_spatial = 0.001  # Weight for the spatial loss

for epoch in range(num_epochs):
    model.train()
    total_loss = 0
    for data, target in train_loader:
        optimizer.zero_grad()
        output = model(data)
        loss = criterion(output, target)
        loss += l1_regularization(model, lambda_l1)

        # Calculate spatial loss for the outputs of the second last layer
        second_last_layer_output = model.dropout(model.bn3(model.fc3(torch.relu(model.bn2(model.fc2(torch.relu(model.bn1(model.fc1(data.view(-1, 32 * 32))))))))))
        sl = spatial_loss(second_last_layer_output)
        loss += lambda_spatial * sl

        loss.backward()
        optimizer.step()
        total_loss += loss.item()

    print(f'Epoch {epoch+1}/{num_epochs}, Loss: {total_loss/len(train_loader)}')

# Testing the model
model.eval()
correct = 0
total = 0

with torch.no_grad():
    for data, target in test_loader:
        output = model(data)
        _, predicted = torch.max(output.data, 1)
        total += target.size(0)
        correct += (predicted == target).sum().item()

print(f'Accuracy: {100 * correct / total}%')


Files already downloaded and verified
Files already downloaded and verified
Epoch 1/50, Loss: 3.7664496349861554
Epoch 2/50, Loss: 3.1731145360585673
Epoch 3/50, Loss: 3.089784645363498


KeyboardInterrupt: 